<a href="https://colab.research.google.com/github/Alirs2002/skimLit-200k-RTC-PubMed/blob/main/skimlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

in this project,I am going to classify pubMed 200k RCT dataset's sentences,into 5 groups

1-objective

2-methods

3-results

4-conclusions

5-background

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

now,lets get the data

In [2]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct.git


Cloning into 'pubmed-rct'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 33 (delta 5), reused 5 (delta 5), pack-reused 25
Receiving objects: 100% (33/33), 177.08 MiB | 23.30 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Updating files: 100% (13/13), done.


In [3]:
!ls

pubmed-rct  sample_data


In [4]:
!cd pubmed-rct/

In [5]:
!ls pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign

dev.txt  test.txt  train.zip


In [6]:
data_dir = "pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign"

In [7]:
import os

file_names = [data_dir + file_name for file_name in os.listdir(data_dir)]
print(file_names)

['pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_signdev.txt', 'pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_signtrain.zip', 'pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_signtest.txt']


In [8]:
!pwd

/content


In [9]:
import zipfile

zipfile = zipfile.ZipFile("./pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign/train.zip")
zipfile.extractall()

In [10]:
!ls

pubmed-rct  sample_data  train.txt


In [11]:
def read_lines(filename):
  listt = []
  with open(filename,"r") as f:
    for line in f:
      listt.append(line)

  return listt


In [12]:
lines = read_lines("train.txt")
print(len(lines))

2593169


now,it is time to create a preprocessing function

In [13]:
def preprocess(filename):

  input_lines = read_lines(filename)

  abstract_lines = ""
  abstract_samples = []


  for line in input_lines:
    if line.startswith("###"):
      abstract_id = line
      abstract_lines = ""
    elif line.isspace():
      abstract_line_split = abstract_lines.splitlines()

      for abstract_line_number, abstract_line in enumerate(abstract_line_split):
        line_data = {}
        target_text_split = abstract_line.split("\t")
        line_data["target"] = target_text_split[0]
        line_data["text"] = target_text_split[1].lower()
        line_data["line_number"] = abstract_line_number
        line_data["total_lines"] = len(abstract_line_split) - 1
        abstract_samples.append(line_data)

    else:
      abstract_lines += line
  return abstract_samples

In [14]:
test_sample_data = preprocess("/content/pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign/test.txt")
train_sample_data = preprocess("/content/train.txt")
validation_sample_data = preprocess("/content/pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign/dev.txt")


In [15]:
train_sample_data[10:15]

[{'target': 'BACKGROUND',
  'text': 'actrn@ .',
  'line_number': 10,
  'total_lines': 10},
 {'target': 'BACKGROUND',
  'text': 'the aim of this study was to evaluate the efficacy , safety and complications of orbital steroid injection versus oral steroid therapy in the management of thyroid-related ophthalmopathy .',
  'line_number': 0,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'a total of @ patients suffering from thyroid ophthalmopathy were included in this study .',
  'line_number': 1,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'patients were randomized into two groups : group i included @ patients treated with oral prednisolone and group ii included @ patients treated with peribulbar triamcinolone orbital injection .',
  'line_number': 2,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'only @ patients in both groups ( @ female and @ male ) completed the study .',
  'line_number': 3,
  'total_lines': 11}]

as you can see,our preprocessing function is working great

In [16]:
# now,lets build a pandas dataframe,for better preformance

In [17]:
len(train_sample_data),len(test_sample_data),len(validation_sample_data)

(2211861, 29493, 28932)

In [18]:
train_data = pd.DataFrame(train_sample_data)
test_data = pd.DataFrame(test_sample_data)
val_data = pd.DataFrame(validation_sample_data)

In [19]:
train_data

,target,text,line_number,total_lines
0,BACKGROUND,the emergence of hiv as a chronic condition me...,0,10
1,BACKGROUND,this paper describes the design and evaluation...,1,10
2,METHODS,this study is designed as a randomised control...,2,10
3,METHODS,the intervention group will participate in the...,3,10
4,METHODS,the program is based on self-efficacy theory a...,4,10
...,...,...,...,...
2211856,METHODS,"after crossover , the wait-list group received...",5,9
2211857,METHODS,performance was assessed by comparison to amer...,6,9
2211858,RESULTS,performance improved significantly after simul...,7,9
2211859,RESULTS,no improvement was detected as a function of c...,8,9


now, i think we should make our own datasets of train,test and validation sentences and train,test and validation labels

In [20]:
# craeting sentences
train_sentence = train_data["text"]
test_sentence = test_data["text"]
val_sentence = val_data["text"]


# creating labels
train_label = train_data["target"]
test_label = test_data["target"]
val_label = val_data["target"]


In [21]:
lengths = np.mean([len(sentence.split()) for sentence in train_sentence])

In [22]:
lengths

26.229355280462922

In [23]:
train_sentence[3],len(train_sentence[3])

("the intervention group will participate in the online group program ` positive outlook ' .",
 90)

In [24]:
sentence_lengths = [len(sentence.split()) for sentence in train_sentence]
sentence_lengths[:10]

[36, 27, 31, 15, 31, 17, 19, 25, 25, 26]

as we can see,98 percent of our sentences,are 67 or shorter

In [25]:
np.percentile(sentence_lengths,96)

57.0

as you can see,the average length of a sentence in our dataset is 26,so our text vectorizer,should be capable of vectorizing this big sentences

our next step is to vectorize our sentences

in the article,it is written that the vocabulary length of this pubmed dataset is about 80000

In [29]:
max_tokens = 70000

vectorizer = tf.keras.layers.TextVectorization(max_tokens=70000,
                                               output_sequence_length=55)

next step is to adapt our vectorizer

In [ ]:
vectorizer.adapt(train_sentence)

In [ ]:
embedding_1  = tf.keras.layers.Embedding(input_dim = max_tokens,
                                         output_dim = 128,
                                         mask_zero = True,
                                         name="kiri")

Now,we should create our labels,in one hot encoded format

In [ ]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder()
y_train_one_hot = one_hot_encoder(train_label)
y_test_one_hot = one_hot_encoder(test_label)
y_val_one_hot = one_hot_encoder(val_label)


Now,lets create our datasets

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_sentence,train_label_one_hot))
val_dataset = tf.data.Dataset.from_tensor_slices((val_sentence,val_label_one_hot))
test_datasset = tf.data.Dataset.from_tensor_slices((test_sentence,test_label_one_hot))

In [ ]:
train_dataset = train_dataset.batch(16).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

and now,it is time to create our model

In [ ]:
input_1  = tf.keras.layers.Input(shape=(1,),dtype="string")
x = vectorizer(input_1)
x = embedding_1(1)
x = tf.keras.layers.Conv1D(128,padding="same",activaiton=tf.keras.activations.relu,kernel_size=2)(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dense(32,activaiton = tf.keras.activations.relu)(x)
output_1 = tf.keras.layers.Dense(5,activation = tf.keras.activations.softmax)(x)

model_1 = tf.keras.Model(input_1,output_1)